# Setup

## Install and import required python libraries
Elastic uses the eland python library to download modesl from Hugging Face hub and load them into elasticsearch

In [3]:
pip -q install eland elasticsearch sentence_transformers transformers torch

In [4]:
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

import getpass

/usr/local/lib/python3.10/dist-packages/eland/ml/_optional.py:116: UserWarning: Eland requires version '1.3' or newer of 'sklearn' (version '1.2.2' currently installed). Use pip or conda to update sklearn.
  warnings.warn(msg, UserWarning)
<ipython-input-4-d329d25d548f>:5: DeprecationWarning: Importing from the 'elasticsearch.client' module is deprecated. Instead use 'elasticsearch' module for importing the client.
  from elasticsearch.client import MlClient


## Configure elasticsearch authentication.
The recommended authentication approach is using the Elastic Cloud ID and a cluster level API key

You can use any method you wish to set the required credentials. We are using getpass in this example to prompt for credentials to avoide storing them in github.

In [5]:
es_cloud_id = getpass.getpass('Enter Elastic Cloud ID:  ')
es_user = getpass.getpass('Enter cluster username:  ')
es_pass = getpass.getpass('Enter cluster password:  ')

Enter Elastic Cloud ID:  ··········
Enter cluster username:  ··········
Enter cluster password:  ··········


## Connect to Elastic Cloud

In [6]:
es = Elasticsearch(cloud_id=es_cloud_id,
                   basic_auth=(es_user, es_pass)
                   )
es.info() # should return cluster info


ObjectApiResponse({'name': 'instance-0000000000', 'cluster_name': 'db3b3840a622415584800a4714e5d6f5', 'cluster_uuid': 'ea9VrQVQQ-q40pjLGRM_rQ', 'version': {'number': '8.10.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'c63272efed16b5a1c25f3ce500715b7fddf9a9fb', 'build_date': '2023-10-05T10:15:55.152563867Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Load the model From Hugging Face into Elasticsearch
Here we specify the model id from Hugging Face. The easiest way to get this id is clicking the copy the model name icon next to the name on the model page.

When calling TransformerModel you specify the HF model id and the task type. You can try specifying auto and eland will attempt to determine the correct type from info in the model config. This is not always possible so a list of specific task_type values can be viewed in the following code: Supported values

In [8]:

# Set the model name from Hugging Face and task type
hf_model_id='sentence-transformers/all-distilroberta-v1'
tm = TransformerModel(model_id=hf_model_id, task_type="text_embedding")

#set the modelID as it is named in Elasticsearch
es_model_id = tm.elasticsearch_model_id()

# Download the model from Hugging Face
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

# Load the model into Elasticsearch
ptm = PyTorchModel(es, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)


  0%|          | 0/312 [00:00<?, ? parts/s]

## Starting the Model
View information about the model
This is not required but can be handy to get a model overivew

In [9]:
# List the in elasticsearch
m = MlClient.get_trained_models(es, model_id=es_model_id)
m.body

{'count': 1,
 'trained_model_configs': [{'model_id': 'sentence-transformers__all-distilroberta-v1',
   'model_type': 'pytorch',
   'created_by': 'api_user',
   'version': '10.0.0',
   'create_time': 1697341427216,
   'model_size_bytes': 0,
   'estimated_operations': 0,
   'license_level': 'platinum',
   'description': "Model sentence-transformers/all-distilroberta-v1 for task type 'text_embedding'",
   'tags': [],
   'input': {'field_names': ['text_field']},
   'inference_config': {'text_embedding': {'vocabulary': {'index': '.ml-inference-native-000001'},
     'tokenization': {'roberta': {'do_lower_case': False,
       'with_special_tokens': True,
       'max_sequence_length': 512,
       'truncate': 'first',
       'span': -1,
       'add_prefix_space': False}}}},
   'location': {'index': {'name': '.ml-inference-native-000001'}}}]}

## Deploy the model
This will load the model on the ML nodes and start the process(es) making it available for the NLP task

In [10]:
s = MlClient.start_trained_model_deployment(es, model_id=es_model_id)
s.body

{'assignment': {'task_parameters': {'model_id': 'sentence-transformers__all-distilroberta-v1',
   'deployment_id': 'sentence-transformers__all-distilroberta-v1',
   'model_bytes': 326154657,
   'threads_per_allocation': 1,
   'number_of_allocations': 1,
   'queue_capacity': 1024,
   'cache_size': '326154657b',
   'priority': 'normal'},
  'routing_table': {'gAvKvqEzRECkJ4hJnbHOSQ': {'current_allocations': 1,
    'target_allocations': 1,
    'routing_state': 'started',
    'reason': ''}},
  'assignment_state': 'started',
  'start_time': '2023-10-15T03:45:23.588288379Z',
  'max_assigned_allocations': 1}}

Verify the model started without issue
Should output -> {'routing_state': 'started'}

In [11]:
stats = MlClient.get_trained_models_stats(es, model_id=es_model_id)
stats.body['trained_model_stats'][0]['deployment_stats']['nodes'][0]['routing_state']

{'routing_state': 'started'}